In [ ]:
%load_ext autoreload
%autoreload 2
%cd '/workspaces/polarization_reddit'

In [ ]:
from preprocessing.utils import load_comments_dask
from load.constants import DATA_PATH
from eda.visualize import barplot_top, plot_daily_comments, plot_daily_sentiment

In [ ]:
YEARS = [2014, 2015, 2016]
comments = {}

In [ ]:
for year in YEARS:
    comments[year] = load_comments_dask(
        DATA_PATH, year=year, tokenize=False, type_file="parquet", dev=False
    )


In [ ]:
for year in YEARS:
    barplot_top(comments[year], column="subreddit", year=year, n=10)

In [ ]:
for year in YEARS:
    barplot_top(comments[year], column="author", year=year, n=500)

In [ ]:
for year in YEARS:
    plot_daily_comments(comments[year], year=year)

In [ ]:
for year in YEARS:
    plot_daily_sentiment(comments[year], year=year)